<a href="https://colab.research.google.com/github/abdishifa234/DS-Unit-2-Kaggle-Challenge/blob/master/module3-cross-validation/LS_DS_223_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 2, Module 3*

---

# Cross-Validation


## Assignment
- [ ] [Review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2), then submit your dataset.
- [ ] Continue to participate in our Kaggle challenge. 
- [ ] Use scikit-learn for hyperparameter optimization with RandomizedSearchCV.
- [ ] Submit your predictions to our Kaggle competition. (Go to our Kaggle InClass competition webpage. Use the blue **Submit Predictions** button to upload your CSV file. Or you can use the Kaggle API to submit your predictions.)
- [ ] Commit your notebook to your fork of the GitHub repo.


**You can't just copy** from the lesson notebook to this assignment.

- Because the lesson was **regression**, but the assignment is **classification.**
- Because the lesson used [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html), which doesn't work as-is for _multi-class_ classification.

So you will have to adapt the example, which is good real-world practice.

1. Use a model for classification, such as [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)
2. Use hyperparameters that match the classifier, such as `randomforestclassifier__ ...`
3. Use a metric for classification, such as [`scoring='accuracy'`](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values)
4. If you’re doing a multi-class classification problem — such as whether a waterpump is functional, functional needs repair, or nonfunctional — then use a categorical encoding that works for multi-class classification, such as [OrdinalEncoder](https://contrib.scikit-learn.org/categorical-encoding/ordinal.html) (not [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html))



## Stretch Goals

### Reading
- Jake VanderPlas, [Python Data Science Handbook, Chapter 5.3](https://jakevdp.github.io/PythonDataScienceHandbook/05.03-hyperparameters-and-model-validation.html), Hyperparameters and Model Validation
- Jake VanderPlas, [Statistics for Hackers](https://speakerdeck.com/jakevdp/statistics-for-hackers?slide=107)
- Ron Zacharski, [A Programmer's Guide to Data Mining, Chapter 5](http://guidetodatamining.com/chapter5/), 10-fold cross validation
- Sebastian Raschka, [A Basic Pipeline and Grid Search Setup](https://github.com/rasbt/python-machine-learning-book/blob/master/code/bonus/svm_iris_pipeline_and_gridsearch.ipynb)
- Peter Worcester, [A Comparison of Grid Search and Randomized Search Using Scikit Learn](https://blog.usejournal.com/a-comparison-of-grid-search-and-randomized-search-using-scikit-learn-29823179bc85)

### Doing
- Add your own stretch goals!
- Try other [categorical encodings](https://contrib.scikit-learn.org/categorical-encoding/). See the previous assignment notebook for details.
- In additon to `RandomizedSearchCV`, scikit-learn has [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Another library called scikit-optimize has [`BayesSearchCV`](https://scikit-optimize.github.io/notebooks/sklearn-gridsearchcv-replacement.html). Experiment with these alternatives.
- _[Introduction to Machine Learning with Python](http://shop.oreilly.com/product/0636920030515.do)_ discusses options for "Grid-Searching Which Model To Use" in Chapter 6:

> You can even go further in combining GridSearchCV and Pipeline: it is also possible to search over the actual steps being performed in the pipeline (say whether to use StandardScaler or MinMaxScaler). This leads to an even bigger search space and should be considered carefully. Trying all possible solutions is usually not a viable machine learning strategy. However, here is an example comparing a RandomForestClassifier and an SVC ...

The example is shown in [the accompanying notebook](https://github.com/amueller/introduction_to_ml_with_python/blob/master/06-algorithm-chains-and-pipelines.ipynb), code cells 35-37. Could you apply this concept to your own pipelines?


### BONUS: Stacking!

Here's some code you can use to "stack" multiple submissions, which is another form of ensembling:

```python
import pandas as pd

# Filenames of your submissions you want to ensemble
files = ['submission-01.csv', 'submission-02.csv', 'submission-03.csv']

target = 'status_group'
submissions = (pd.read_csv(file)[[target]] for file in files)
ensemble = pd.concat(submissions, axis='columns')
majority_vote = ensemble.mode(axis='columns')[0]

sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission[target] = majority_vote
submission.to_csv('my-ultimate-ensemble-submission.csv', index=False)
```

In [ ]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [ ]:
import pandas as pd

# Merge train_features.csv & train_labels.csv
train = pd.merge(pd.read_csv(DATA_PATH+'waterpumps/train_features.csv'), 
                 pd.read_csv(DATA_PATH+'waterpumps/train_labels.csv'))

# Read test_features.csv & sample_submission.csv
test = pd.read_csv(DATA_PATH+'waterpumps/test_features.csv')
sample_submission = pd.read_csv(DATA_PATH+'waterpumps/sample_submission.csv')

In [ ]:
#lets arrange and merge train,test submission datasets

train = pd.merge(pd.read_csv(DATA_PATH+'waterpumps/train_features.csv'), 
                 pd.read_csv(DATA_PATH+'waterpumps/train_labels.csv'))
test = pd.read_csv(DATA_PATH+'waterpumps/test_features.csv')
sample_submission = pd.read_csv(DATA_PATH+'waterpumps/sample_submission.csv')

train.shape, test.shape

((59400, 41), (14358, 40))

In [ ]:
#lets import the tools here
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import category_encoders as ce
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import f_regression, SelectKBest
from sklearn.preprocessing import StandardScaler

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
# Split training set into train & val sets
train, val = train_test_split(train, train_size=0.80, test_size=0.20, 
                              stratify=train['status_group'], random_state=42)

train.shape, val.shape, test.shape

((47520, 41), (11880, 41), (14358, 40))

In [ ]:
#define a function that rangles all the 3 datasets
def wrangle(X):
    """Wrangle train, validate, and test sets in the same way"""
    
    # Prevent SettingWithCopyWarning
    X = X.copy()
    
    # About 3% of the time, ladef wrangle(X):titude has small values near zero,
    # outside Tanzania, so we'll treat these values like zero.
    X['latitude'] = X['latitude'].replace(-2e-08, 0)
    
    # When columns have zeros and shouldn't, they are like null values.
    # So we will replace the zeros with nulls, and impute missing values later.
    # Also create a "missing indicator" column, because the fact that
    # values are missing may be a predictive signal.
    cols_with_zeros = ['longitude', 'latitude', 'construction_year','gps_height', 'population']
    for col in cols_with_zeros:
        X[col] = X[col].replace(0, np.nan)
        X[col+'_MISSING'] = X[col].isnull()
            
    # Drop duplicate columns
    duplicates = ['quantity_group', 'payment_type']
    X = X.drop(columns=duplicates)
    
    # Drop recorded_by (never varies) and id (always varies, random)
    unusable_variance = ['recorded_by']
    X = X.drop(columns=unusable_variance)
    
    # Convert date_recorded to datetime
    X['date_recorded'] = pd.to_datetime(X['date_recorded'], infer_datetime_format=True)
    
    # Extract components from date_recorded, then drop the original column
    X['year_recorded'] = X['date_recorded'].dt.year
    X['month_recorded'] = X['date_recorded'].dt.month
    X['day_recorded'] = X['date_recorded'].dt.day
    X = X.drop(columns='date_recorded')
    
    # Engineer feature: how many years from construction_year to date_recorded
    X['years'] = X['year_recorded'] - X['construction_year']
    X['years_MISSING'] = X['years'].isnull()

    #creat water per person and population per year

    X['water/person'] = X['amount_tsh'] / X['population']

    X['pop/year'] = X['population'] / X['year_recorded']

    
    # return the wrangled dataframe
    return X


# Wrangle train, validate, and test sets in the same way
train = wrangle(train)
val = wrangle(val)
test = wrangle(test)

In [ ]:
# Arrange data into X features matrix and y target vector
target = 'status_group'
X_train = train.drop(columns=target)
y_train = train[target]
X_val = val.drop(columns=target)
y_val = val[target]
X_test = test

In [ ]:
#lets fit and check our accuracy
%%time
pipeline = make_pipeline(
    ce.OrdinalEncoder(),
    SimpleImputer(strategy='most_frequent'),
    RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=20, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=30, min_samples_split=4,
                       min_weight_fraction_leaf=0.0, n_estimators=135,
                       n_jobs=-1, oob_score=True, random_state=42, verbose=0,
                       warm_start=False)
)

#fitting on train data
pipeline.fit(X_train, y_train)

#predicting on testing data
y_pred = pipeline.predict(X_test)

#scoring on validation data
print ('Train Accuracy', pipeline.score(X_train, y_train))
print ('Validation Accuracy', pipeline.score(X_val, y_val))

Train Accuracy 0.7957491582491583
Validation Accuracy 0.7761784511784512
CPU times: user 24.2 s, sys: 162 ms, total: 24.3 s
Wall time: 14.7 s


In [ ]:
#lets search the best parametrs using Randomized Search cross validation
from scipy.stats import randint, uniform
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV


features = train.columns.drop(target)
X_train = train[features]
y_train = train[target]

pipeline = make_pipeline(
    ce.OrdinalEncoder(),
    SimpleImputer(), 
    RandomForestClassifier(n_jobs=-1, random_state=12, max_features='auto', 
                             oob_score=True, criterion='gini')
)

param_distributions = {
    'simpleimputer__strategy': ['mean', 'median', 'most_frequent'], 
    'randomforestclassifier__n_estimators': [130, 135, 140],
    'randomforestclassifier__max_depth': [22, 23, 25, 26], 
    'randomforestclassifier__min_samples_split': [5, 6], 
}

# If you're on Colab, decrease n_iter & cv parameters
search = RandomizedSearchCV(
    pipeline, 
    param_distributions=param_distributions, 
    n_iter=24, 
    cv=5, 
    scoring='accuracy', 
    verbose=50, 
    return_train_score=True, 
    n_jobs=-1,
    random_state=42
)

search.fit(X_train, y_train);

Fitting 5 folds for each of 24 candidates, totalling 120 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   28.9s
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   29.1s
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:   55.6s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:   55.9s
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done  11 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:  3

In [ ]:
#show the best hyperparametrs and cross validation MAE
print('Best hyperparameters', search.best_params_)
print('Cross-validation MAE', -search.best_score_)

Best hyperparameters {'simpleimputer__strategy': 'mean', 'randomforestclassifier__n_estimators': 130, 'randomforestclassifier__min_samples_split': 5, 'randomforestclassifier__max_depth': 22}
Cross-validation MAE -0.8064814814814815


In [ ]:
# Display cv results by ranking the test scores
final_search = pd.DataFrame(search.cv_results_)
final_search.sort_values(by='rank_test_score').T

,3,9,11,10,13,22,14,5,20,1,6,12,15,21,4,2,17,18,23,8,16,7,19,0
mean_fit_time,21.6857,21.5655,22.4057,23.3715,21.3675,24.8863,23.852,23.9551,24.3482,25.7255,21.4173,21.2093,22.5258,22.9636,21.5964,21.8043,22.889,22.8723,24.6949,21.9575,23.7216,23.3184,24.2648,23.1743
std_fit_time,0.357917,0.662839,0.389678,0.634405,0.363863,0.270269,0.335091,0.321749,0.484984,0.518626,0.426242,0.509362,0.501149,0.526791,0.479974,0.309217,0.253116,0.560994,0.651014,0.428705,0.774701,0.426371,2.42625,0.227267
mean_score_time,0.793221,0.867985,0.875322,0.907926,0.931033,0.874439,0.926392,0.97954,0.958629,0.87898,0.878918,0.931544,0.929963,0.896804,0.822551,0.832455,0.882023,0.91517,0.827803,0.878774,0.954675,0.906132,0.968814,0.816549
std_score_time,0.0840296,0.105174,0.101488,0.0766919,0.0720893,0.103788,0.091608,0.065392,0.0458225,0.10944,0.0765777,0.036902,0.0302141,0.0997347,0.126805,0.111431,0.0102267,0.0688307,0.219334,0.0800989,0.0327172,0.122192,0.145604,0.0874594
param_simpleimputer__strategy,mean,mean,median,most_frequent,mean,most_frequent,most_frequent,most_frequent,most_frequent,most_frequent,median,mean,median,median,median,mean,median,mean,most_frequent,mean,most_frequent,median,mean,median
param_randomforestclassifier__n_estimators,130,130,135,130,130,140,135,135,135,140,130,130,135,140,130,130,135,140,140,135,130,140,135,135
param_randomforestclassifier__min_samples_split,5,5,6,6,6,6,5,6,5,5,6,6,5,6,6,5,6,6,5,6,5,6,5,5
param_randomforestclassifier__max_depth,22,26,23,25,25,23,22,25,26,26,22,22,23,22,23,23,25,23,25,22,26,23,25,22
params,"{'simpleimputer__strategy': 'mean', 'randomfor...","{'simpleimputer__strategy': 'mean', 'randomfor...","{'simpleimputer__strategy': 'median', 'randomf...","{'simpleimputer__strategy': 'most_frequent', '...","{'simpleimputer__strategy': 'mean', 'randomfor...","{'simpleimputer__strategy': 'most_frequent', '...","{'simpleimputer__strategy': 'most_frequent', '...","{'simpleimputer__strategy': 'most_frequent', '...","{'simpleimputer__strategy': 'most_frequent', '...","{'simpleimputer__strategy': 'most_frequent', '...","{'simpleimputer__strategy': 'median', 'randomf...","{'simpleimputer__strategy': 'mean', 'randomfor...","{'simpleimputer__strategy': 'median', 'randomf...","{'simpleimputer__strategy': 'median', 'randomf...","{'simpleimputer__strategy': 'median', 'randomf...","{'simpleimputer__strategy': 'mean', 'randomfor...","{'simpleimputer__strategy': 'median', 'randomf...","{'simpleimputer__strategy': 'mean', 'randomfor...","{'simpleimputer__strategy': 'most_frequent', '...","{'simpleimputer__strategy': 'mean', 'randomfor...","{'simpleimputer__strategy': 'most_frequent', '...","{'simpleimputer__strategy': 'median', 'randomf...","{'simpleimputer__strategy': 'mean', 'randomfor...","{'simpleimputer__strategy': 'median', 'randomf..."
split0_test_score,0.808291,0.808186,0.808817,0.808502,0.807029,0.808607,0.807449,0.806713,0.807134,0.808291,0.807449,0.806503,0.806923,0.806082,0.807449,0.806608,0.805029,0.806082,0.808607,0.805976,0.80766,0.808502,0.807765,0.806187


In [ ]:
#using the best parametrs above lets fit and check our val accuracy
pipeline = make_pipeline(
    ce.OrdinalEncoder(),
    SimpleImputer(strategy='mean'),
    RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=22, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=20, min_samples_split=5,
                       min_weight_fraction_leaf=0.0, n_estimators=130,
                       n_jobs=-1, oob_score=True, random_state=42, verbose=0,
                       warm_start=False)
)

pipeline.fit(X_train, y_train)
print ('Train Accuracy', pipeline.score(X_train, y_train))
print ('Validation Accuracy', pipeline.score(X_val, y_val))

Train Accuracy 0.8117845117845118
Validation Accuracy 0.7868686868686868


In [ ]:
# prep your submission file
print(len(test))
print(len(y_pred))
print(y_pred[:10])

14358
14358
['non functional' 'functional' 'functional' 'non functional' 'functional'
 'functional' 'functional' 'non functional' 'non functional' 'functional']


In [ ]:
submission=pd.DataFrame(y_pred, columns=['status_group'])
submission['id']=test['id']
submission=submission[['id', 'status_group']]
submission.head()

,id,status_group
0,50785,non functional
1,51630,functional
2,17168,functional
3,45559,non functional
4,49871,functional


In [ ]:
submission.to_csv('submission_file.csv',index=False)
submission.shape

(14358, 2)